In [1]:
from util import make_document

DIR_PATH = "chinese_data/"

documents = make_document(DIR_PATH)
documents

['《Arcaea》 （中國大陸官方譯為「韻律源點」 ）是 Lowiro開發的一款三維立體音\n樂遊戲。遊戲的首個版本於 2017年3月9日在App Store 與Google Play 發\n布。2023年2月23日，Arcaea下載量突破 1200萬。 \n',
 '任天堂Switch版本的Arcaea於2021年5月18日上線，支持觸控和 Joy-Con\n兩種操作方式。  \n',
 '遊戲系統  \n物件類型  \nArcaea擁有4種物件類型，分別為 Tap，Hold，ArcTap和Arc，遊戲目標在於\n在物件落到判定面上時接住這些物件。  \n',
 '其中Tap和Hold只會出現在地面的軌道上， ArcTap和Arc有梯形範圍限制\n（存在少數譜面的部分物件超出這一限制） 。  \n',
 '遊戲判定  \n對於Tap和ArcTap來說，按照其點擊時刻與預期時刻的誤差由小到大可分為以\n下3類：PURE，FAR和LOST。 \n',
 '對於Hold和Arc來說，其本質為數個判定點單獨判定。若在判定點到達判定面\n時有符合要求的手指在判定區間內，則獲得 PURE判定，反之獲得 LOST判\n定。  \n',
 'Arc不允許換手。同時，在不開視覺輔助時， Arc的默認手序為左手接藍色\nArc，右手接紅色 Arc。 \n',
 '譜面難度 /定數  遊戲包含五個難度，分別為： Past，Present，Future，Eternal和Beyond。在\n遊戲內上述五個難度會分別簡稱為 PST，PRS，FTR，ETR和BYD。 \n',
 '一般情況下，每首曲目均擁有 Past，Present，Future難度譜面各一張，而少數\n曲目在移動端擁有 Beyond或Eternal譜面。Beyond譜面可能使用不同的音源\n和/或曲繪，也存在少數曲目的 Past，Present，Future難度曲繪有不同。絕大\n多數曲目的各難度按照 Past，Present，Future，Eternal/Beyond 順序難度遞\n增。  \n',
 '每張譜面均有難度標註，範圍為 1~12級。越高的等級標註一般意味著譜面的\n難度更大。  \n',
 '在計算潛力值時，會代入細化難度的定數值以計算。定數最多精確至一位小\n數。部分低等級的定數劃分檔位相對較少，同時部分

In [2]:
from sentence_transformers import SentenceTransformer

# 加載模型
embbeded_model = SentenceTransformer('thenlper/gte-large-zh')

c:\Users\weiso131\anaconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
document_ebeddings = embbeded_model.encode(documents)

C:\Users\weiso131\AppData\Roaming\Python\Python312\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 使用者查詢
user_query = "介紹Arcaea的難度分級"

# 查詢文本轉換成嵌入向量
query_embedding = embbeded_model.encode([user_query])

# 計算相似度
similarities = cosine_similarity(query_embedding, document_ebeddings)

# 找到最相似的文本
most_similar_idx = np.argmax(similarities)
retrieved_answers = documents[most_similar_idx]
print("最相似的文本:", retrieved_answers)
print("相似度分數:", similarities[0][most_similar_idx])

最相似的文本: 每張譜面均有難度標註，範圍為 1~12級。越高的等級標註一般意味著譜面的
難度更大。  

相似度分數: 0.69903904


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 加載模型和tokenizer
model_name = "fnlp/bart-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [7]:
input_text = f"前後文:{retrieved_answers}\n\n 提問:{user_query}\n\n 答案:"

# 將文本轉換為模型可讀的格式
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

# 移除 'token_type_ids' (BART 不需要這個)
inputs.pop('token_type_ids', None)
# 生成結果
outputs = model.generate(**inputs, max_length=50, num_beams=5, early_stopping=True)

# 解碼生成的文本
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("生成的文本:", generated_text)

生成的文本: 前 後 文 : 每 張 譜 面 均 有 難 度 標 註 ， 範 圍 為 1 ~ 12 級 難 度 越 高 的 等 級 標 註 一 般 意 味 著 譜 面 的 難 度 更 大 的 提 問 :
